In [ ]:
!pip install -q pypdf
# !pip install -q python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip install torch
!pip install -q transformers
!pip -q install sentence-transformers

In [ ]:
!pip install -q llama-index

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 51.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.36-cp310-cp310-manylinux_2_35_x86_64.whl size=9412113 sha256=cd06ec85f16f52a9ecaac1036f821bf1306e260ba07f4300c79b4d94b82c55af
  Stored in directory: /tmp/pip-ephem-wheel-cache-86jg2rav/wheels/26/0e/4a/94c586df881726eea841c3780236351cdad7bef2303c7607a7
Successfully built llama-cpp-python


In [ ]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=4096,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)


total size (MB): 4368.44


4167it [00:23, 178.55it/s]                          
AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.chat_template': "{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] + eos_token}}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}", 'tokenizer.ggml.add_eos_token': 'false', 'tokenizer.ggml.padding_token_id': '0', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.rope.freq_base': '1000000.

In [ ]:
from llama_index import SimpleDirectoryReader
documents = SimpleDirectoryReader(
    input_files = ["./documents/survey_on_llms.pdf"]
).load_data()

In [ ]:
from llama_index import Document
documents = Document(text = "\n\n".join([doc.text for doc in documents]))

In [ ]:
type(documents)

llama_index.schema.Document

In [ ]:
# Putting all it together
import os
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index import VectorStoreIndex, ServiceContext
def get_build_index(documents,llm,embed_model="local:BAAI/bge-small-en-v1.5",sentence_window_size=3,save_dir="./vector_index/index"):

  node_parser = SentenceWindowNodeParser(
      window_size = sentence_window_size,
      window_metadata_key = "window",
      original_text_metadata_key = "original_text"
  )

  sentence_context = ServiceContext.from_defaults(
      llm = llm,
      embed_model= embed_model,
      node_parser = node_parser,
  )

  if not os.path.exists(save_dir):
        # create and load the index
        index = VectorStoreIndex.from_documents(
            [documents], service_context=sentence_context
        )
        index.storage_context.persist(persist_dir=save_dir)
  else:
      # load the existing index
      index = load_index_from_storage(
          StorageContext.from_defaults(persist_dir=save_dir),
          service_context=sentence_context,
      )

  return index


In [ ]:
def get_query_engine(sentence_index, similarity_top_k=6, rerank_top_n=2):
  postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
  rerank = SentenceTransformerRerank(
      top_n=rerank_top_n, model="BAAI/bge-reranker-base"
  )
  engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
  )

  return engine


In [ ]:
# get the vector index
vector_index = get_build_index(documents=documents, llm=llm, embed_model="local:BAAI/bge-small-en-v1.5", sentence_window_size=3, save_dir="./vector_index/index")

In [ ]:
query_engine = get_query_engine(sentence_index=vector_index, similarity_top_k=6, rerank_top_n=2)

In [ ]:
from llama_index.response.notebook_utils import display_response

In [ ]:
while True:
  query=input()
  response = query_engine.query(query)
  display_response(response)
  print("\n")

Llama.generate: prefix-match hit


**`Final Response:`** Large Language Models (LLMs) refer to Transformer language models that contain hundreds of billions or more parameters, trained on massive text data. They exhibit strong capacities to understand natural language and solve complex tasks through text generation. LLMs have emerged as a significant advancement in Natural Language Processing (NLP), with pre-trained language models being scaled up to achieve improved model capacity and special abilities such as in-context learning. The research community uses the term "large language models" for PLMs of significant size, and recent progress in LLMs has led to the development of powerful AI chatbots like ChatGPT, which has attracted widespread attention from society.

Llama.generate: prefix-match hit


**`Final Response:`** A large language model (LLM) refers to Transformer language models that contain hundreds of billions or more parameters, trained on massive text data. These models exhibit strong capacities to understand natural language and solve complex tasks through text generation. They have been largely advanced by both academia and industry, with notable examples being GPT-3, PaLM, Galactica, and LLaMA. LLMs have shown significant performance improvements over smaller models and exhibit special abilities such as in-context learning. The term "large language model" was coined to distinguish these models from smaller pre-trained language models (PLMs). Recent advancements in LLMs have had a significant impact on the AI community, leading to innovations like ChatGPT, a powerful AI chatbot developed based on LLMs.

Llama.generate: prefix-match hit


...

**`Final Response:`** Yes, LLaMA2 is an open-source large language model. The context information indicates that it is developed based on the deep learning framework MindSpore and has attracted significant attention from the research community with many efforts being devoted to fine-tuning or continually pre-training its different model versions.

Llama.generate: prefix-match hit


**`Final Response:`** Yes, LLaMA2 is an open-source large language model (LLM). The context information indicates that it is developed based on the deep learning framework MindSpore and has attracted significant attention from the research community with many efforts being devoted to fine-tuning or continually pre-training its different model versions.

Llama.generate: prefix-match hit


**`Final Response:`** Based on the provided context, there is no mention or indication that Mistral-7B is an LLM (Language Model) or if it is open-source.